# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

/tmp/tmpoqasdag5/fbb7ad1854f8455286e2cedd40b3312212c7f17a/treevalue/tree/integration/torch.py:18: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  register_for_torch(TreeValue)
/tmp/tmpoqasdag5/fbb7ad1854f8455286e2cedd40b3312212c7f17a/treevalue/tree/integration/torch.py:19: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  register_for_torch(FastTreeValue)


In [3]:
t

<FastTreeValue 0x7f8fecfed4c0>
├── 'a' --> tensor([[-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581]])
└── 'x' --> <FastTreeValue 0x7f8fecfedfa0>
    └── 'c' --> tensor([[-1.6102,  0.4350,  0.7629,  0.2790],
                        [ 0.7358,  0.7413,  0.7137, -0.8552],
                        [-0.2541, -0.9803, -0.1087, -0.1179]])

In [4]:
t.a

tensor([[-0.9624,  0.8154,  0.0285],
        [ 1.8705,  0.2981, -0.9581]])

In [5]:
%timeit t.a

48 ns ± 0.182 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8fecfed4c0>
├── 'a' --> tensor([[ 0.6973,  0.3593,  2.5663],
│                   [-0.9195, -0.8220, -0.0599]])
└── 'x' --> <FastTreeValue 0x7f8fecfedfa0>
    └── 'c' --> tensor([[-1.6102,  0.4350,  0.7629,  0.2790],
                        [ 0.7358,  0.7413,  0.7137, -0.8552],
                        [-0.2541, -0.9803, -0.1087, -0.1179]])

In [7]:
%timeit t.a = new_value

51.7 ns ± 0.219 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581]]),
    x: Batch(
           c: tensor([[-1.6102,  0.4350,  0.7629,  0.2790],
                      [ 0.7358,  0.7413,  0.7137, -0.8552],
                      [-0.2541, -0.9803, -0.1087, -0.1179]]),
       ),
)

In [10]:
b.a

tensor([[-0.9624,  0.8154,  0.0285],
        [ 1.8705,  0.2981, -0.9581]])

In [11]:
%timeit b.a

41.9 ns ± 0.553 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6658,  1.1796, -2.7990],
               [ 0.1809, -0.4904, -1.4357]]),
    x: Batch(
           c: tensor([[-1.6102,  0.4350,  0.7629,  0.2790],
                      [ 0.7358,  0.7413,  0.7137, -0.8552],
                      [-0.2541, -0.9803, -0.1087, -0.1179]]),
       ),
)

In [13]:
%timeit b.a = new_value

355 ns ± 0.433 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

624 ns ± 4.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

8.46 µs ± 22.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

128 µs ± 634 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

127 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8f059378e0>
├── 'a' --> tensor([[[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]],
│           
│                   [[-0.9624,  0.8154,  0.0285],
│                    [ 1.8705,  0.2981, -0.9581]]])
└── 'x' --> <FastTreeValue 0x7f8fecff3100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

23.6 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8efd42e880>
├── 'a' --> tensor([[-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581],
│                   [-0.9624,  0.8154,  0.0285],
│                   [ 1.8705,  0.2981, -0.9581]])
└── 'x' --> <FastTreeValue 0x7f8ff4424d90>
    └── 'c' --> tensor([[-1.6102,  0.4350,  0.7629,  0.2790],
                        [ 0.7358,  0.7413,  0.7137, -0.8552],
                 

In [23]:
%timeit t_cat(trees)

22.2 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

50.9 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]],
       
               [[-0.9624,  0.8154,  0.0285],
                [ 1.8705,  0.2981, -0.9581]]]),
    x: Batch(
           c: tensor([[[-1.6102,  0.4350,  0.7629,  0.2790],
                       [ 0.7358,  0.7413,  0.7137, -0.8552],
                       [-0.2541, -0.9803, -0.1087, -0.1179]],
         

In [26]:
%timeit Batch.stack(batches)

62.5 µs ± 513 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581],
               [-0.9624,  0.8154,  0.0285],
               [ 1.8705,  0.2981, -0.9581]]),
    x: Batch(
           c: tensor([[-1.6102,  0.4350,  0.7629,  0.2790],
                      [ 0.7358,  0.7413,  0.7137, -0.8552],
                      [-0.2541, -0.9803, -0.1087, -0.1179],
                      [-1.6102,  0.4350,  0.7629,  0.2790],
                      [ 0.7358,  

In [28]:
%timeit Batch.cat(batches)

117 µs ± 724 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

275 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
